Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "Ilya Grebnekin"
COLLABORATORS = "-"

---

# Derivatives: Finite difference schemes

Write a function, `deriv`, which computes a derivative of its argument at a given point, $x$, using a finite difference rule with a given step side $h$, with the approximation order of $O(h^2)$ (e.g., use a central difference scheme). 

In [1]:
def deriv(f, x, h):
    """ Compute a derivative of `f` at point `x` with step size `h`.
    
    Compute the derivative using the symmetric scheme.
    
    Parameters
    ----------
    f : callable
        The function to differentiate
    x : float
        The point to compute the derivative at.
    h : float
        The step size for the finite different rule.
        
    Returns
    -------
    fder : derivative of f(x) at point x using the step size h.
    """

    fder = (f(x + h) - f(x - h)) / (2 * h)

    return fder

Test your function on a simple test case: differentiate $f(x) = x^3$ at $x=0$. Are your results consistent with the expected value of $f'(x) = 0$? Are they consistent with the expected scaling of the error with $h\to 0$?

In [2]:
x = 0
for h in [1e-2, 1e-3, 1e-4, 1e-5]:
    err = deriv(lambda x: x ** 3, x, h)
    print("%5f -- %7.4g" % (h, err))

0.010000 --  0.0001
0.001000 --   1e-06
0.000100 --   1e-08
0.000010 --   1e-10


In [3]:
from numpy.testing import assert_allclose

assert_allclose(deriv(lambda x: x ** 3, 0, h=1e-4),
                0, atol=1e-6)
assert_allclose(deriv(lambda x: x ** 4, 1, h=1e-7),
                4, atol=1e-6)

from math import log

assert_allclose(deriv(lambda x: x ** 2 * log(x), 1, h=1e-5),
                1, atol=1e-6)

## One-sided finite difference schemes

Now implement two one-sided finite difference schemes for the first derivative: a two-point forward difference and a three-point forward difference. 

Test your functions on $f(x) = x^2 \log{x}$ at $x = 1$.
Study the dependence of the error with $h$. Roughly estimate the value of the step size $h$ where the error stops decreasing.  While the error still decreses with $h$, what is the scaling of the error, is it $O(h)$ or $O(h^2)$?

In [4]:
from math import log


def func(x):
    return x ** 2 * log(x)


def deriv_forward_2pt(f, x, h):
    """Estimate $df/dx$ at x using a two-point forward difference scheme."""

    fder = (f(x + h) - f(x)) / h

    return fder


def deriv_forward_3pt(f, x, h):
    """Estimate $df/dx$ at x using a three-point forward difference scheme."""

    fder = (-3 * f(x) + 4 * f(x + h) - f(x + 2 * h)) / (2 * h)

    return fder


In [5]:
# Test your functions

assert_allclose(deriv_forward_2pt(func, 1, h=1e-5),
                1, atol=1e-4)

assert_allclose(deriv_forward_3pt(func, 1, h=1e-5),
                1, atol=1e-6)

In [16]:
# deriv_forward_2pt consistent with the O(h) and deriv_forward_3pt consistent with the O(h^2) scaling of the error

c3, c2 = 0, 0
h = 1
for i in range(1, 100):

    h_i = h / i ** 5
    h_1 = h / (i + 1) ** 5
    h_2 = h / (i + 2) ** 5

    if abs(deriv_forward_2pt(func, 1, h_2) - deriv_forward_2pt(func, 1, h_1)) > abs(
            deriv_forward_2pt(func, 1, h_1) - deriv_forward_2pt(func, 1, h_i)) and c2 == 0:
        print('for deriv2 i =', i + 1, 'h =', h_1)
        c2 = 1

    if abs(deriv_forward_3pt(func, 1, h_2) - deriv_forward_3pt(func, 1, h_1)) > abs(
            deriv_forward_3pt(func, 1, h_1) - deriv_forward_3pt(func, 1, h_i)) and c3 == 0:
        print('for deriv3 i =', i + 1, 'h =', h_1)
        c3 = 1


for deriv3 i = 12 h = 4.018775720164609e-06
for deriv2 i = 31 h = 3.492943259127733e-08


### One-sided differences at a boundary

Now try differentiating $x^2 \log(x)$ at $x=0$. Use the three-point one-sided rule. Note that to evaluate the function at zero, you need to special-case this value. Check the scaling of the error with $h$, explain your results. 

In [10]:
def f(x):
    if x == 0:
        # the limit of $x^2 log(x)$ at $x-> 0$ is zero, even though log(x) is undefined at x=0
        return 0.0
    else:
        return x ** 2 * log(x)


x = 0
for h in [1e-2, 1e-3, 1e-4, 1e-5]:
    der = deriv_forward_3pt(f, x, h)
    print("%5f -- %7.4g" % (h, der))

0.010000 -- -0.01386
0.001000 -- -0.001386
0.000100 -- -0.0001386
0.000010 -- -1.386e-05


...

In [11]:
assert_allclose(deriv_forward_3pt(f, 0, h=1e-5),
                0, atol=1e-3)